# Problem set 6: Solving the Solow model 

In [ ]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# Tasks

## Solving matrix equations I

In [ ]:
np.random.seed(1900)
n = 5
A = np.random.uniform(size=(n,n))
b = np.random.uniform(size=n)
c = np.random.uniform(size=n)
d = np.random.uniform(size=n)

**Question A:** Find the determinant of $[A \cdot A]^{-1}$

In [ ]:
# write your code here
np.linalg.det(np.linalg.inv(A@A)) ##<----- @ is matrix multiplication

**Answer:** see A1.py

**Question B:** Solve the following equation systems directly using **scipy**.

$$
\begin{aligned}
Ax &= b \\
Ax &= c \\
Ax &= d 
\end{aligned}
$$

In [ ]:
print(linalg.solve(A, b))
print(linalg.solve(A, c))
print(linalg.solve(A, d))

In [ ]:
## this approach works but is slow... This is beacue it relies on matrix inversion each time it solves an equation
%timeit linalg.solve(A,b); linalg.solve(A, c); linalg.solve(A, d)

**Answer:** A2.py

**Question C:** Solve the same equation systems as above using `linalg.lu_factor()` and `linalg.lu_solve()`. What is the benefit of this approach?

In [ ]:
lu, piv = linalg.lu_factor(A)

print(linalg.lu_solve((lu, piv), b))
print(linalg.lu_solve((lu, piv), c))
print(linalg.lu_solve((lu, piv), d))

In [ ]:
## much faster - no matrix inversion + we can reuse the LU factorization between the three systems
%timeit lu, piv = linalg.lu_factor(A); linalg.lu_solve((lu, piv), b); linalg.lu_solve((lu, piv), c); linalg.lu_solve((lu, piv), d)

In [ ]:
## this is especially beneficial if we solve several equations using the same A!

**Answer:** A3.py

## Solving matrix equations II

In [ ]:
F = np.array([[2.0, 1.0, -1.0], [-3.0, -1.0, 2], [-2.0, 1.0, 2.0]])
e = np.array([8.0, -11.0, -3.0])

**Question:** Use the function `gauss_jordan()` in the `numecon_linalg` module located in this folder to solve

$$
Fx = e
$$

In [ ]:
# write your code here
import numecon_linalg as nl

Fe = np.column_stack((F,e)) ## Concatenate matrix and vector
nl.gauss_jordan(Fe) ## Fe is updated in place
x = Fe[:,-1] ## last column of reduced matrix is solution
print(x) 

**Answer:** see A4.py

## Symbolic

**Question A:** Find

$$
\lim_{x \rightarrow 0} \frac{\sin(x)}{x}
$$

and

$$
\frac{\partial\sin(2x)}{\partial x} 
$$

In [ ]:
# write your code here
x = sm.symbols('x')
f1 = sm.sin(x)/x
print('limit:', sm.limit(f1,x,0))

f2 = sm.sin(2*x)
print('derivative:',sm.diff(f2))

**Question B:** Solve the equation

$$ 
\frac{\sin(x)}{x} = 0
$$

In [ ]:
# write your code here
sm.solve(f1, x) ## one solution

In [ ]:
## solveset can find the set of solutios
sm.solveset(f1, x)

**Answer:** A6.py

# Problem: Solve the Solow model

## Introduction

Consider the **standard Solow-model** where:

1. $K_t$ is capital2
2. $L_t$ is labor (growing with a constant rate of $n$)
3. $A_t$ is technology (growing with a constant rate of $g$)
4. $Y_t = F(K_t,A_tL_t)$ is GDP

**Saving** is a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},1)\equiv f(\tilde{k}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio.

The **transition equation** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}_{t})+(1-\delta)\tilde{k}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t},A_{t}L_{t})=K_{t}^{\alpha}(A_{t}L_{t})^{1-\alpha}\Rightarrow f(\tilde{k}_{t})=\tilde{k}_{t}^{\alpha}
$$

If it is **CES** (with $\beta < 1, \beta \neq 0$) then

$$
F(K_{t},A_{t}L_{t})=(\alpha K_{t}^{\beta}+(1-\alpha)(A_{t}L_{t})^{\beta})^{\frac{1}{\beta}}\Rightarrow f(\tilde{k}_{t})=(\alpha\tilde{k}_{t}^{\beta}+(1-\alpha))^{\frac{1}{\beta}}
$$

## Steady state

Assume the production function is **Cobb-Douglas**.

**Question A:** Use **sympy** to find an analytical expression for the steady state, i.e. solve

$$
\tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}]
$$

In [ ]:
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

In [ ]:
f = k**alpha                                ## define production function
k_next = 1/((1+n)*(1+g)) * (s*f + (1-delta)*k)  ## transition in k
k_ss = sm.solve(k_next-k, k)[0]             ## solve for steady state
k_ss

**Answer:** see A7.py

**Question B:** Turn you solution into a Python function called as `ss_func(s,g,n,delta,alpha)`. 

In [ ]:
ss_func = sm.lambdify((s,g,n,delta,alpha), k_ss) ## use sm.lambdify to turn k_ss into function 

## we can now use it to evaluate ss as function of parameters
ss_func(0.2, 0.02, 0.01, 0.1, 1/3)

**Answer:** A8.py

**Question C**: Find the steady state numerically using root-finding with `optimize.root_scalar`.

In [ ]:
s = 0.2
g = 0.02
n = 0.01
alpha = 1/3
delta = 0.1

## I create a simple namespace to hold parameters for convenience
from types import SimpleNamespace
par = SimpleNamespace()
par.s = s
par.g = g
par.n = n
par.alpha = alpha
par.delta = delta

## production function
def f_CD(k,par):
    return k**par.alpha

## transition
def k_next(k, f, par): ## note how I give the production funciton f as input 
    return 1/((1+par.n)*(1+par.g)) * (par.s*f(k, par) + (1-par.delta)*k)

## find steady state
def find_ss(par, f, bracket=(0.1, 10)):
    # objective: k_next - k = 0
    obj = lambda k: k_next(k, f, par) - k
    res = optimize.root_scalar(obj, bracket=bracket) 
    assert res.converged
    return res.root

## call the steady state function (with production function as input)
ss_CD = find_ss(par, f_CD)
print(f'root finder: {ss_CD:5.4f}')

## does this align with the symbolic solution?
print(f'sympy: {ss_func(s,g,n,delta,alpha):5.4f}')

**Answer:** A9.py

**Question D:** Now assume the production function is CES. Find the steady state for $k$ for the various values of $\beta$ shown below.

In [ ]:
betas = [-0.5,-0.25,-0.1,-0.05,0.05,0.1,0.25,0.5]

# write your code here
def f_CES(k, par):
    return (par.alpha*k**par.beta + (1-par.alpha))**(1/par.beta)

ks = []

## loop through list of betas
for beta in betas:
    par.beta = beta ## update beta
    k_ss = find_ss(par, f_CES) ## I can just use the steady state function from earlier and update the produciton function
    ks.append(k_ss)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_axes(111)
ax.plot(betas, ks)
ax.set_xlabel('$\\beta$')
ax.set_ylabel('$k^*$')
ax.grid(True)

## So: increasing beta (substitutability) increases effective capital per worker

## Sanity check: when beta = 0, CES reduces to Cobb-Douglas. Lets check that this aligns with our findings from the previous problem
ax.scatter(0, ss_CD, label='Cobb Douglass production')
ax.legend()

In [ ]:
## could we have solved the CES-model with sympy?
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')
beta = sm.symbols('beta')

f = (alpha*k**beta + (1-alpha))**(1/beta)                ## new production function
k_next = 1/((1+n)*(1+g)) * (s*f + (1-delta)*k)  
# k_ss = sm.solve(k_next-k, k)[0] ## <---------------- try uncommenting this line

**Answer:** A10.py   